# NCSU GIS 582: Geospatial Modeling and Analysis

## Tutorial: 2 - Geospatial Data Models and Visualization

### Resources

- [GRASS Manual](https://grass.osgeo.org/grass-devel/manuals/)
- [GRASS Jupyter Introduction](https://grass.osgeo.org/grass-devel/manuals/jupyter_intro.html)
- [GRASS Python API Introduction](https://grass.osgeo.org/grass-devel/manuals/python_intro.html)

## Google Colab Setup

Let’s first print system description to know where are we.

In [ ]:
!lsb_release -a

At the time of writing this tutorial, Colab has Linux Ubuntu 22.04.5 LTS. So we add the ppa:ubuntugis repository, update and install GRASS. It might take a couple of minutes according to the resources available.

In [ ]:
!add-apt-repository -y ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt-get install -y grass-core grass-dev

Check that GRASS is installed by asking which version is there.

In [ ]:
!grass --version

Check which Python version is running.

In [ ]:
import sys

v = sys.version_info
print(f"We are using Python {v.major}.{v.minor}.{v.micro}")

## GRASS Setup

Import the Python standard libraries we need.

In [ ]:
import subprocess
import os
from pathlib import Path

We are going to import the GRASS Python API (`grass.script`) and the GRASS Jupyter package (`grass.jupyter`), but first, we need to find the path to those packages using the `--config python_path` command. This command is slightly different for each operating system.

We use `subprocess.check_output` to find the path and `sys.path.append` to add it to the path.

In [ ]:
# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

In [ ]:
# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

Download and unzip the North Carolina basic sample dataset:

> A complete list of available datasets can be found at: https://grass.osgeo.org/download/data/

If you are running this tutorial locally, please change the path to where you have unzipped the sample dataset.

In [ ]:
!grass --tmp-project XY --exec g.download.project url=https://grass.osgeo.org/sampledata/north_carolina/nc_spm_08_grass7.tar.gz path=/content

Start a GRASS session using the sample dataset.

In [ ]:
# Start GRASS Session
session = gj.init(Path("nc_spm_08_grass7", "user1"))

Now that we have initialized the GRASS session, we can start using GRASS tools and functions.

## 2A: Geospatial Data Models

### 1. Resampling raster data to different resolutions

Resample the given raster map to higher and lower resolution (30m->10m, 30m->100m) and compare resampling by nearest neighbor with bilinear and bicubic method.

First, set the computation region extent to our study area and set resolution to 30 meters. The computational region (region for short) is set using g.region module, the -p flag is used to print the region coordinates and resolution. Here we use a predefined region which is included in our data set. 

In [ ]:
!g.region region=swwake_30m -p

Then we display the 30m resolution NED elevation raster. To display raster maps in Jupyter Notebooks, we can use the `gj.Map` function from the `grass.jupyter` Python package.

> By default, the map will be saved in the current working directory. You can specify a different path by providing the full path in the `filename` parameter. In Google Colab, we save the output images in the `/content` directory. However you can change it to any other directory you prefer or mount your Google Drive to save the outputs there by including teh following code at the beginning of the notebook:

```python
from google.colab import drive
drive.mount('/content/drive')
```

For more information, please refer to [this guide](https://colab.research.google.com/notebooks/io.ipynb).

In [ ]:
# Create Map instance
elev_ned_30m_map = gj.Map(filename="elev_ned_30m.png")
elev_ned_30m_map.d_rast(map="elev_ned_30m")

# Display map
elev_ned_30m_map.show()

To resample it to 10m resolution, first set the computational region to resolution 10m, then resample the raster using the nearest neighbor method.

> How many new cells do you expect in the resampled raster compared to the original raster?

In [ ]:
!g.region res=10 -p
!r.resamp.interp elev_ned_30m out=elev_ned10m_nn method=nearest

Now display the resampled map "elev_ned10m_nn" the same way we displayed the original map.

In [ ]:
# Add you code below to display the resampled map elev_ned10m_nn

In [ ]:
# @title Solution

# Create Map instance
m = gj.Map()
m.d_rast(map="elev_ned10m_nn")

# Display map
m.show()

The elevation map "elev_ned10m_nn" looks the same as the original one, so now check the resampled elevation surface using the aspect map: 

In [ ]:
!r.slope.aspect elevation=elev_ned10m_nn aspect=aspect_ned10m_nn

 Display the aspect map derived from the resampled elevation map.

#### Map: Aspect Nearest Neighbor Interoplation

In [ ]:
# Create Map instance
aspect_ned10m_nn_map = gj.Map(filename="aspect_nn.png")
aspect_ned10m_nn_map.d_rast(map="aspect_ned10m_nn")

# Display map
aspect_ned10m_nn_map.show()

Now, reinterpolate DEMs using bilinear and bicubic interpolation. Check the structure of interpolated elevation surfaces using aspect maps. 

In [ ]:
!r.resamp.interp elev_ned_30m out=elev_ned10m_bil meth=bilinear
!r.resamp.interp elev_ned_30m out=elev_ned10m_bic meth=bicubic
!r.slope.aspect elevation=elev_ned10m_bil aspect=aspect_ned10m_bil
!r.slope.aspect elevation=elev_ned10m_bic aspect=aspect_ned10m_bic

#### Map: Aspect Bilinear Interoplation

In [ ]:
# Create Map instance
aspect_ned10m_bil_map = gj.Map(filename="aspect_bil.png")
aspect_ned10m_bil_map.d_rast(map="aspect_ned10m_bil")

# Display map
aspect_ned10m_bil_map.show()

#### Map: Aspect Bicubic Interoplation

In [ ]:
# Create Map instance
aspect_ned10m_bic_map = gj.Map(filename="aspect_bic.png")
aspect_ned10m_bic_map.d_rast(map="aspect_ned10m_bic")

# Display map
aspect_ned10m_bic_map.show()

#### Question 1

**Why is the aspect map of the elevation raster map computed with the nearest neighbor method different from the one computed by bilinear interpolation?**

Resample to lower resolution (30m -> 100m).

Now change the region's resolution and resample elevation (which is a continuous field) and land use (which has discrete categories).

In [ ]:
!g.region res=100 -p
!r.resamp.stats elev_ned_30m out=elev_new100m_avg method=average

#### Map: Elevation 100m

In [ ]:
# Create Map instance
elev_100m_map = gj.Map(filename="elev_100m.png")
elev_100m_map.d_rast(map="elev_new100m_avg")

# Display map
elev_100m_map.show()

Resample landuse to 100m resolution

In [ ]:
!r.resamp.stats landuse96_28m out=landuse96_100m method=mode

#### Map: Landuse 100m

In [ ]:
# Create Map instance
landuse96_100m_map = gj.Map(filename="landuse96_100m.png")
landuse96_100m_map.d_rast(map="landuse96_100m")

# Display map
landuse96_100m_map.show()

#### Question 2

**Explain selection of aggregation method. Can we use average also for landuse? What does mode mean?**

### 2. Convert from vector to raster

Now let's convert vector data to raster for use in raster-based analysis.

Convert the polylines in the "streets" vector to raster. Set the resolution to 30m and use speed limit attribute.

In [ ]:

!g.region res=30 -p
!v.to.rast streets_wake out=streets_speed_30m use=attr attrcol=SPEED type=line

#### Map: Vector to Raster

In [ ]:
# Create Map instance
vect_to_rast_map = gj.Map(filename="vect_to_rast.png")

# Display raster data
vect_to_rast_map.d_rast(map="streets_speed_30m")

# Display raster legend
# vect_to_rast_map.d_legend(raster="streets_speed_30m", at=[5,30,2,5], use=[25,35,45,55,65] )
# Add legend
# Full documentation found at:
# https://grass.osgeo.org/grass-devel/manuals/d.legend.html
vect_to_rast_map.d_legend(
    raster="streets_speed_30m",
    at=[10, 45, 5, 12],
    fontsize=12,
    title="Speed Limit (mph)",
    title_fontsize=14,
    color="#000000",
    use=[25, 35, 45, 55, 65],
    flags="b",
)

# Add a scale bar and north arrow
# Full documentation found at:
# https://grass.osgeo.org/grass-devel/manuals/d.barscale.html
vect_to_rast_map.d_barscale(
    at=[2, 8],
    style="both_ticks",
    color="#000000",
    bgcolor="none",
    fontsize=14,
    flags="n",
)

# Display map
vect_to_rast_map.show()


### 3. Convert from raster to vector data

Let's convert DEM derived streams to vector lines. First, set the computational region to the streams raster map (`streams_derived`), then [thin](https://grass.osgeo.org/grass-devel/manuals/r.thin.html) the raster lines and convert to vector lines.

In [ ]:
!g.region raster=streams_derived -p
!r.thin streams_derived output=streams_derived_t
!r.to.vect streams_derived_t output=streams_derived_t type=line

#### Question 3

**Explain why we are using the r.thin tool.**

 Visually compare the result with streams digitized from airphotos.

#### Map: Streams Compare

In [ ]:
# Create Map instance
streams_compare_map = gj.Map(filename="streams_compare.png")

# Display vector data
streams_compare_map.d_vect(map="streams_derived_t", color="blue", fill_color=None, legend_label="Derived Streams")
streams_compare_map.d_vect(map="streams", color="red", legend_label="Digitized Streams")

# Display raster legend
streams_compare_map.d_legend_vect(at=[70,12], flags="b")

# Display map
streams_compare_map.show()

Convert raster areas representing basins to vector polygons.

To convert, use raster values as category numbers (flag -v) and display boundaries of vector polygons.
Verify the basin boundaries by displaying them together with streams - the stream networks should "fit" within the basin boundaries.

In [ ]:
!g.region raster=basin_50K -p
!r.to.vect -sv basin_50K output=basin_50Kval type=area

#### Map: Basins

In [ ]:
# Create Map instance
basins_map = gj.Map(filename="basins.png")

# Display raster data
basins_map.d_rast(map="basin_50K")

# Display vector data
basins_map.d_vect(map="basin_50Kval", type="boundary", width=2)
basins_map.d_vect(map="streams", color="blue")

# Display map
basins_map.show()

## 2B: Data display and visualization

#### Changing the default font

Change the default font used for when rendering a 2D map by setting the font with `gj.Map(font="Font_Name")` or set the font independently for individual map elements like raster, vector, legend, etc.

### 1. Basic 2D display operations

Visualy explore relation between developed areas and topography. Set the region and display land use categories 1, 2 (developed land) over shaded topography.

Set the computational region to landuse map.

In [ ]:
!g.region raster=landuse96_28m -p

#### Map: Developed Area

In [ ]:
# Create Map instance
landuse_elev_map = gj.Map(filename="mylandsat.png")

landuse_elev_map.d_rgb(red="lsat7_2002_30", green="lsat7_2002_20", blue="lsat7_2002_10")

landuse_elev_map.d_rast(
    map="landuse96_28m",
    values=[1,2]
)

# Add Major Roads
landuse_elev_map.d_vect(map="roadsmajor", color="yellow")

# Display map
landuse_elev_map.show()

#### Question 1

**How did the developed area change between 1996 and 2002?**

### 2. Change colors for raster maps

There are many ways how to adjust or create custom color ramps for raster maps, see [r.colors](https://grass.osgeo.org/grass-devel/manuals/r.colors.html) manual, we explore only some basic tools here.

Compare the use of equal interval and histogram equalized color table for slope

First, we will create our copy of the slope map.

In [ ]:
!g.copy raster=slope,myslope

Now display it with an equal interval color ramp with colors ranging from blue-green-yellow to red (bgyr).

In [ ]:
!r.colors myslope color=bgyr

#### Map: Slope Equal Interval Color Ramp

In [ ]:
# Create Map instance
myslope_map = gj.Map(filename="myslopecolor.png")

# Display raster data
myslope_map.d_rast(map="myslope")

# Add Legend
myslope_map.d_legend(
    raster="myslope",
    at=[10, 45, 5, 12],
    fontsize=12,
    title="Slope (degrees)",
    title_fontsize=14,
    color="#000000",
    flags="b"
)

# Display map
myslope_map.show()

Change to the histogram equalized color table, and save the new slope map.

In [ ]:
!r.colors -e myslope color=bgyr

In [ ]:
# Create Map instance
myslope_map = gj.Map(filename="myslopecolorequalized.png")

# Display raster data
myslope_map.d_rast(map="myslope")

# Add Legend
myslope_map.d_legend(
    raster="myslope",
    at=[10, 45, 5, 12],
    fontsize=12,
    title="Slope (degrees)",
    title_fontsize=14,
    color="#000000",
    flags="b",
)

# Display map
myslope_map.show()


To explain the difference between the two maps, you can generate a histogram.

#### Histogram: Slope Color Histogram Equalized

In [ ]:
slopehistogram = gj.Map(filename="slopehistogram.png")
slopehistogram.d_histogram(map="myslope")
slopehistogram.show()

#### Question 2

**What is the effect of the histogram equalized color table on the slope map pattern?**

### 3. Modify legend, scale and grid

To re-size the legend for myslope you need to update the `at` parameter of `d.legend`.

```python
myslope_map.d_legend(
    raster="myslope",
    at=[10, 45, 5, 12],
    fontsize=12,
    title="Slope (degrees)",
    title_fontsize=14,
    color="#000000",
    flags="b",
)
```

The numbers are bottom,top,left,right as percentage of screen coordinates.

#### Task: Update Slope Map Components

1. Add units to the [legend](https://grass.osgeo.org/grass-devel/manuals/d.legend.html)
2. Add [barscale](https://grass.osgeo.org/grass-devel/manuals/d.barscale.html) and change its length and units.

> Note: you can use horizontal legends by using Placement at=6,10,2,30

In [ ]:
# @title Solution to Task: Update Slope Map Components

# Create Map instance
myslope_map = gj.Map()

# Display raster data
myslope_map.d_rast(map="myslope")
myslope_map.d_barscale(length=1000)
myslope_map.d_legend(raster="myslope", at=[6,10,20,50])

# Display map
myslope_map.show()

Add grid for state plane coordinates at 5000m with ticks at 1000m. Also add a lat/long grid at 2 arc minute interval.

#### Map: Grid for state plane coordinates at 5000m with ticks at 1000m.

In [ ]:
# Create Map instance
myslope_map = gj.Map(filename="myslopemap_gridxy.png")

# Display raster data
myslope_map.d_rast(map="myslope")
myslope_map.d_grid(size=5000, color="brown")
myslope_map.d_grid(size=1000, flags="n")
myslope_map.d_barscale(at=[2,8], style="both_ticks", color="#000000", bgcolor="none", fontsize=14, flags="n")
myslope_map.d_legend(
    raster="myslope",
    at=[10, 45, 5, 12],
    fontsize=12,
    title="Slope (degrees)",
    title_fontsize=14,
    color="#000000",
    flags="b",
)

# Display map
myslope_map.show()

#### Map: Grid lat/long grid at 2 arc minute interval

In [ ]:
# Create Map instance
myslope_map = gj.Map(filename="myslopemap_gridlatlong.png")

# Display raster data
myslope_map.d_rast(map="myslope")
myslope_map.d_grid(size="0:02", color="black", flags="g")
myslope_map.d_barscale(
    at=[2, 8],
    style="both_ticks",
    color="#000000",
    bgcolor="none",
    fontsize=14,
    flags="n",
)
myslope_map.d_legend(
    raster="myslope",
    at=[10, 45, 5, 12],
    fontsize=12,
    title="Slope (degrees)",
    title_fontsize=14,
    color="#000000",
    flags="b",
)

# Display map
myslope_map.show()


### 4. Visualization in 3D perspective

GRASSs' native 3D visualization tool for Jupyter Notebooks does not currently work in a Google Colab environment. However, you can run the following code locally in your Jupyter Notebook environment where GRASS is installed to generate a 3D elevation map.

```python
elevation_3dmap = gj.Map3D(filename="3dElevation.png")
# Full list of options m.nviz.image
# https://grass.osgeo.org/grass83/manuals/m.nviz.image.html
elevation_3dmap.render(
    elevation_map="elevation",
    color_map="elevation",
    perspective=20,
    height=3000,
    vline="streams",
    vline_color="blue",
    fringe=['ne','nw','sw','se'],
    arrow_position=[100,50],
)
elevation_3dmap.overlay.d_legend(raster="elevation", at=(60, 97, 87, 92))
elevation_3dmap.show()
```

As a work around for Google Colab, you can use tools like [PyVista](https://docs.pyvista.org/) to create 3D visualizations of geospatial data. The following example using PyVista to visualize a DEM raster.

First we need to export the elevation raster to a vtk file using the `r.out.vtk` GRASS tool.

In [ ]:
!g.region raster=elevation
!r.out.vtk -c input=elevation output=elevation.vtk elevation=elevation zscale=10

Now we can insatall PyVista and the required dependencies Mesa for offscreen rendering in Colab.

In [ ]:
!apt-get install -qq xvfb libgl1-mesa-glx
!pip install pyvista -qq

In [ ]:
import pyvista as pv

pv.global_theme.jupyter_backend = "static"

# Load the VTK file
terrain = pv.read("elevation.vtk")
plotter = pv.Plotter(notebook=True)
plotter.add_light(pv.Light(position=(1000, 1000, 1000), intensity=0.8))
plotter.add_light(pv.Light(position=(-1000, -1000, 1000), intensity=0.5))
plotter.add_mesh(terrain, opacity=1.0, nan_opacity=0.0, smooth_shading=True)
plotter.show(auto_close=False)
plotter.close()